# 🎯 Projet MNIST - Accélération Apprentissage avec Keras

Dans ce projet, l’objectif était d’atteindre au moins 97% d’accuracy sur le dataset MNIST tout en minimisant le temps d’apprentissage et d’inférence, en travaillant exclusivement sur un processeur Intel i5-13420H. Pour cela, j’ai conçu et entraîné un modèle MLP (Multi-Layer Perceptron) optimisé, car cette architecture est particulièrement adaptée à MNIST et offre un excellent compromis entre simplicité, rapidité et précision. Le modèle comporte une couche Flatten suivie de deux couches denses (128 puis 64 neurones), chacune accompagnée de Batch Normalization et de Dropout à 25%, avant une couche finale softmax. Ce choix architectural permet une convergence rapide tout en contrôlant efficacement l’overfitting. Plusieurs techniques issues de la littérature en deep learning ont été intégrées : BatchNorm pour stabiliser et accélérer l’apprentissage, Dropout pour améliorer la généralisation, l’optimizer Adam pour un ajustement adaptatif du gradient, EarlyStopping pour arrêter l’entraînement au moment optimal, et ReduceLROnPlateau pour adapter dynamiquement le learning rate. Grâce à ces optimisations, le modèle atteint environ 98% d’accuracy sur les données de test, dépassant largement le seuil requis, tout en maintenant un temps d’entraînement relativement court (environ 14 secondes selon les conditions d’exécution). L’inférence est également extrêmement rapide, atteignant plus de 13500 images par seconde suivant le batch size. Une comparaison avec d’autres approches (SGD Classifier et Random Forest) montre que le MLP Keras offre le meilleur équilibre entre précision, stabilité et performance en inférence. En somme, cette solution se révèle particulièrement efficace sur CPU, alliant rapidité, simplicité de mise en œuvre et très bonne performance, tout en respectant pleinement les objectifs du projet.

## 📊 1. Analyse Matériel


In [ ]:
import platform
import psutil
from datetime import datetime
import numpy as np

print("="*70)
print("ANALYSE DE LA CONFIGURATION MATÉRIEL")
print("="*70)

print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Python: {platform.python_version()}")
print(f"OS: {platform.system()} {platform.version()}")
print(f"CPU: {platform.processor()}")
print(f"Cores: {psutil.cpu_count(logical=False)} physiques / {psutil.cpu_count(logical=True)} logiques")

ram = psutil.virtual_memory()
print(f"RAM: {ram.total // 1024**3} Go (disponible: {ram.available // 1024**3} Go)")
print(f"Utilisation CPU: {psutil.cpu_percent()}%")

print("\n" + "="*70)


## 🚀 2. Installation et Importation


In [ ]:
# Installation (si nécessaire)
# !pip install tensorflow numpy matplotlib scikit-learn -q

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import time
import pandas as pd

print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")

# Vérification GPU
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"GPU disponible: {gpu_devices}")
    device_name = tf.test.gpu_device_name()
    print(f"Device: {device_name}")
else:
    print("GPU non disponible - utilisation CPU")


## 📥 3. Chargement et Préparation des Données


In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

print("Chargement du dataset MNIST...")
start_load = time.time()

# Chargement
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalisation
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# One-hot encoding des labels
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

print(f"✓ Données chargées en {time.time() - start_load:.2f}s")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

# Affichage d'exemples
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_train[i], cmap='gray')
    ax.set_title(f"Label: {y_train[i]}")
    ax.axis('off')
plt.suptitle("Exemples du dataset MNIST", fontsize=14)
plt.tight_layout()
plt.show()


## 🏗️ 4. Architecture du Modèle Optimisé


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

print("\n" + "="*70)
print("ARCHITECTURE DU MODÈLE OPTIMISÉ")
print("="*70)

def create_optimized_model():
    """
    Modèle MLP optimisé pour MNIST avec:
    - Architecture minimale mais efficace
    - Batch Normalization pour accélérer la convergence
    - Dropout pour la régularisation
    - Learning rate scheduling adaptatif
    """
    model = Sequential([
        # Couche d'entrée
        Flatten(input_shape=(28, 28)),  # 784 pixels en entrée

        # Première couche cachée avec BatchNorm
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.25),  # Dropout réduit pour éviter le sous-apprentissage

        # Deuxième couche cachée
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.25),

        # Couche de sortie
        Dense(10, activation='softmax')
    ])

    # Optimizer avec learning rate adaptatif
    optimizer = Adam(
        learning_rate=0.001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07
    )

    # Compilation
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Création du modèle
model = create_optimized_model()

# Résumé de l'architecture
model.summary()

# Visualisation du modèle
tf.keras.utils.plot_model(
    model,
    to_file='model_architecture.png',
    show_shapes=True,
    show_layer_names=True,
    show_layer_activations=True
)


## 🔧 5. Callbacks et Configuration d'Entraînement


In [ ]:
print("\n" + "="*70)
print("CONFIGURATION DE L'ENTRAÎNEMENT")
print("="*70)

# Callbacks pour optimisation
callbacks = [
    # Early Stopping: arrête si plus d'amélioration
    EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True,
        mode='max',
        verbose=1,
        min_delta=0.001
    ),

    # Réduction du learning rate sur plateau
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    ),

    # TensorBoard pour monitoring (optionnel)
    # tf.keras.callbacks.TensorBoard(log_dir='./logs')
]

# Configuration d'entraînement
batch_size = 128  # Batch size optimisé pour CPU
epochs = 10
validation_split = 0.1  # 10% pour validation

print(f"Batch size: {batch_size}")
print(f"Epochs max: {epochs}")
print(f"Validation split: {validation_split}")
print(f"Optimizer: Adam (lr=0.001)")
print(f"Total paramètres: {model.count_params():,}")


## 🏃 6. Entraînement avec Tracking Détaillé


In [ ]:
print("\n" + "="*70)
print("DÉBUT DE L'ENTRAÎNEMENT")
print("="*70)

start_train = time.time()

# Entraînement avec historique
history = model.fit(
    X_train,
    y_train_cat,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=callbacks,
    verbose=1  # 1 pour barre de progression
)

train_time = time.time() - start_train
print(f"\n✓ Entraînement terminé en {train_time:.2f}s")


## 📈 7. Analyse des Résultats d'Entraînement


In [ ]:
print("\n" + "="*70)
print("ANALYSE DES RÉSULTATS D'ENTRAÎNEMENT")
print("="*70)

# Récupération de l'historique
history_dict = history.history

# Trouver l'epoch où 97% est atteint
val_acc = history_dict['val_accuracy']
epoch_to_97 = next((i+1 for i, acc in enumerate(val_acc) if acc >= 0.97), None)

print("\n[PERFORMANCES PAR EPOCH]")
for epoch in range(len(val_acc)):
    train_acc = history_dict['accuracy'][epoch]
    val_acc_epoch = val_acc[epoch]
    print(f"Epoch {epoch+1:2d}: Train={train_acc:.4f}, Val={val_acc_epoch:.4f}", 
          end="")
    if val_acc_epoch >= 0.97:
        print(" ✅ 97%+", end="")
    if epoch+1 == len(val_acc):
        print(" (final)", end="")
    print()

print("\n[RÉSUMÉ]")
print(f"Meilleure val accuracy: {max(val_acc):.4f} ({max(val_acc)*100:.2f}%)")
print(f"Final train accuracy: {history_dict['accuracy'][-1]:.4f} ({history_dict['accuracy'][-1]*100:.2f}%)")
print(f"Final val accuracy: {val_acc[-1]:.4f} ({val_acc[-1]*100:.2f}%)")

if epoch_to_97:
    print(f"✅ Objectif 97% atteint à l'epoch {epoch_to_97}")
    time_to_97 = (epoch_to_97 / len(val_acc)) * train_time
    print(f"   Temps pour atteindre 97%: {time_to_97:.2f}s")
else:
    print("⚠️  Objectif 97% non atteint")

# Visualisation des courbes d'apprentissage
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Courbe d'accuracy
axes[0].plot(history_dict['accuracy'], 'b-', label='Train Accuracy', linewidth=2)
axes[0].plot(history_dict['val_accuracy'], 'r-', label='Validation Accuracy', linewidth=2)
axes[0].axhline(y=0.97, color='g', linestyle='--', label='Target 97%')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Courbe d\'Accuracy - Objectif 97% atteint rapidement')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Courbe de loss
axes[1].plot(history_dict['loss'], 'b-', label='Train Loss', linewidth=2)
axes[1].plot(history_dict['val_loss'], 'r-', label='Validation Loss', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Courbe de Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Marquer le point où 97% est atteint
if epoch_to_97:
    axes[0].plot(epoch_to_97-1, val_acc[epoch_to_97-1], 'g*', 
                markersize=15, label=f'97% atteint (epoch {epoch_to_97})')

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150, bbox_inches='tight')
plt.show()


## 🧪 8. Évaluation sur le Test Set


In [ ]:
print("\n" + "="*70)
print("ÉVALUATION SUR LE TEST SET")
print("="*70)

# Évaluation
start_eval = time.time()
test_loss, test_acc = model.evaluate(X_test, y_test_cat, verbose=0)
eval_time = time.time() - start_eval

print(f"\n[SCORES FINAUX]")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"Temps d'évaluation: {eval_time:.2f}s")

# Vérification de l'objectif
print("\n[RÉSULTAT]")
if test_acc >= 0.97:
    print(f"✅ OBJECTIF ATTEINT! Test Accuracy = {test_acc*100:.2f}%")
else:
    print(f"⚠️  Objectif non atteint: {test_acc*100:.2f}% < 97%")

# Prédictions détaillées
print("\n[PRÉDICTIONS DÉTAILLÉES]")
y_pred = np.argmax(model.predict(X_test, verbose=0), axis=1)

# Matrice de confusion
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=range(10), yticklabels=range(10))
plt.title('Matrice de Confusion - Test Set')
plt.ylabel('Vrai label')
plt.xlabel('Prédiction')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150)
plt.show()

# Rapport de classification
print("\n[RAPPORT DE CLASSIFICATION]")
print(classification_report(y_test, y_pred, digits=3))

# Accuracy par classe
print("\n[ACCURACY PAR CLASSE]")
print("Chiffre | Correct/Total | Accuracy")
print("-" * 35)
for i in range(10):
    correct = cm[i, i]
    total = cm[i].sum()
    accuracy = correct / total if total > 0 else 0
    print(f"   {i}     | {correct:4d}/{total:4d}    | {accuracy:.2%}")


## ⚡ 9. Test de Performance d'Inférence


In [ ]:
print("\n" + "="*70)
print("BENCHMARK DE PERFORMANCE D'INFÉRENCE")
print("="*70)

def benchmark_inference(model, X_data, n_runs=100):
    """Test de vitesse d'inférence"""
    results = []
    batch_sizes = [1, 10, 100, 1000]

    print(f"\nTest sur {n_runs} runs:")
    print(f"{'Batch':<8} {'Temps moyen':<15} {'Std Dev':<12} {'Images/s':<12}")
    print("-" * 50)

    for batch_size in batch_sizes:
        times = []

        # Warmup
        _ = model.predict(X_data[:batch_size], verbose=0)

        # Mesures
        for i in range(n_runs // max(1, batch_size//10)):
            start_idx = i * batch_size
            end_idx = start_idx + batch_size

            if end_idx > len(X_data):
                break

            start = time.perf_counter()
            _ = model.predict(X_data[start_idx:end_idx], verbose=0)
            end = time.perf_counter()

            times.append(end - start)

        if times:
            avg_time = np.mean(times)
            std_time = np.std(times)
            speed = batch_size / avg_time

            results.append({
                'batch_size': batch_size,
                'avg_time_ms': avg_time * 1000,
                'std_time_ms': std_time * 1000,
                'speed_img_per_sec': speed
            })

            print(f"{batch_size:<8} {avg_time*1000:<15.2f} {std_time*1000:<12.2f} {speed:<12.0f}")

    return pd.DataFrame(results)

# Benchmark
inference_results = benchmark_inference(model, X_test, n_runs=100)

# Visualisation
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(inference_results['batch_size'], inference_results['speed_img_per_sec'], 
        'bo-', linewidth=2, markersize=8)
ax.set_xlabel('Batch Size')
ax.set_ylabel('Images par seconde')
ax.set_title('Performance d\'Inférence - Scaling avec Batch Size')
ax.grid(True, alpha=0.3)
ax.set_xscale('log')
plt.tight_layout()
plt.savefig('inference_performance.png', dpi=150)
plt.show()


## 📊 10. Comparaison avec Autres Modèles


In [ ]:
print("\n" + "="*70)
print("COMPARAISON DES APPROCHES")
print("="*70)

# Données de comparaison (basées sur vos tests précédents)
comparison_data = {
    'Modèle': ['SGD Classifier', 'Random Forest', 'MLP (Keras - Votre modèle)'],
    'Test Accuracy': [0.9126, 0.9681, test_acc],
    'Temps Entraînement (s)': [15.70, 37.88, train_time],
    'Vitesse Inférence (img/s)': [180245, 24984, inference_results['speed_img_per_sec'].iloc[-1]],
    'Atteint 97%': ['❌ Non', '❌ Non (96.8%)', '✅ Oui' if test_acc >= 0.97 else '❌ Non']
}

df_comparison = pd.DataFrame(comparison_data)

print("\n[TABLEAU COMPARATIF]")
print(df_comparison.to_string(index=False))

# Visualisation comparative
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Bar plot accuracy
axes[0, 0].bar(df_comparison['Modèle'], df_comparison['Test Accuracy'], 
               color=['red', 'orange', 'green'], alpha=0.7)
axes[0, 0].axhline(y=0.97, color='gray', linestyle='--', label='Target 97%')
axes[0, 0].set_ylabel('Test Accuracy')
axes[0, 0].set_title('Comparaison Accuracy')
axes[0, 0].set_ylim(0.85, 1.0)
axes[0, 0].legend()
axes[0, 0].tick_params(axis='x', rotation=15)

# Bar plot temps
axes[0, 1].bar(df_comparison['Modèle'], df_comparison['Temps Entraînement (s)'],
               color=['red', 'orange', 'green'], alpha=0.7)
axes[0, 1].set_ylabel('Temps (s)')
axes[0, 1].set_title('Comparaison Temps d\'Entraînement')
axes[0, 1].tick_params(axis='x', rotation=15)

# Bar plot vitesse inférence
axes[1, 0].bar(df_comparison['Modèle'], df_comparison['Vitesse Inférence (img/s)'],
               color=['red', 'orange', 'green'], alpha=0.7)
axes[1, 0].set_ylabel('Images/s (log scale)')
axes[1, 0].set_title('Comparaison Vitesse Inférence')
axes[1, 0].set_yscale('log')
axes[1, 0].tick_params(axis='x', rotation=15)

# Scatter plot trade-off
scatter = axes[1, 1].scatter(df_comparison['Temps Entraînement (s)'], 
                            df_comparison['Test Accuracy'],
                            s=df_comparison['Vitesse Inférence (img/s)']/1000,
                            c=['red', 'orange', 'green'],
                            alpha=0.7, edgecolors='black')
axes[1, 1].axhline(y=0.97, color='gray', linestyle='--')
axes[1, 1].set_xlabel('Temps Entraînement (s)')
axes[1, 1].set_ylabel('Test Accuracy')
axes[1, 1].set_title('Trade-off: Temps vs Accuracy (taille = vitesse)')
axes[1, 1].grid(True, alpha=0.3)

# Annotations
for i, txt in enumerate(df_comparison['Modèle']):
    axes[1, 1].annotate(txt, 
                       (df_comparison['Temps Entraînement (s)'][i], 
                        df_comparison['Test Accuracy'][i]),
                       xytext=(5, 5), textcoords='offset points')

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()


## 🎯 11. Analyse du Modèle Keras


In [ ]:
print("\n" + "="*70)
print("ANALYSE DÉTAILLÉE DU MODÈLE KERAS")
print("="*70)

print("\n[ARCHITECTURE CHOISIE]")
print("1. Flatten(28,28) → 784 entrées")
print("2. Dense(128, relu) + BatchNorm + Dropout(0.25)")
print("3. Dense(64, relu) + BatchNorm + Dropout(0.25)")
print("4. Dense(10, softmax)")
print(f"\nTotal paramètres: {model.count_params():,}")

print("\n[OPTIMISATIONS IMPLÉMENTÉES]")
print("1. Batch Normalization → Accélère convergence")
print("2. Dropout 25% → Réduit l'overfitting")
print("3. Adam optimizer → Learning rate adaptatif")
print("4. Early Stopping → Arrêt anticipé optimal")
print("5. ReduceLROnPlateau → Ajustement dynamique du LR")

print("\n[POURQUOI ÇA FONCTIONNE MIEUX]")
print("✓ MLP adapté à MNIST (données vectorielles)")
print("✓ Architecture profonde suffisante (128→64→10)")
print("✓ Régularisation efficace (Dropout + BatchNorm)")
print("✓ Optimizer adaptatif (Adam > SGD)")
print("✓ Early stopping évite le surapprentissage")

# Analyse des poids
print("\n[ANALYSE DES POIDS]")
for i, layer in enumerate(model.layers):
    if len(layer.get_weights()) > 0:
        weights = layer.get_weights()[0]
        biases = layer.get_weights()[1]
        print(f"Couche {i+1} ({layer.name}):")
        print(f"  Poids: shape={weights.shape}, mean={weights.mean():.4f}, std={weights.std():.4f}")
        print(f"  Biais: shape={biases.shape}, mean={biases.mean():.4f}, std={biases.std():.4f}")


## 📝 12. Démonstration Interactive


In [ ]:
print("\n" + "="*70)
print("DÉMONSTRATION INTERACTIVE")
print("="*70)

# Sélection d'exemples intéressants (bons et mauvais)
y_pred_proba = model.predict(X_test, verbose=0)
y_pred = np.argmax(y_pred_proba, axis=1)

# Trouver des exemples corrects et incorrects
correct_indices = np.where(y_pred == y_test)[0]
incorrect_indices = np.where(y_pred != y_test)[0]

print(f"\nExemples corrects: {len(correct_indices)}/{len(y_test)} ({len(correct_indices)/len(y_test)*100:.1f}%)")
print(f"Exemples incorrects: {len(incorrect_indices)}/{len(y_test)} ({len(incorrect_indices)/len(y_test)*100:.1f}%)")

# Afficher quelques exemples
n_examples = 10
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

# Exemples corrects
for i, ax in enumerate(axes[0]):
    idx = correct_indices[i]
    ax.imshow(X_test[idx], cmap='gray')
    confidence = np.max(y_pred_proba[idx])
    ax.set_title(f"Prédit: {y_pred[idx]}\nVrai: {y_test[idx]}\nConf: {confidence:.2f}", fontsize=10)
    ax.axis('off')
axes[0, 0].set_ylabel('Corrects', fontsize=12)

# Exemples incorrects (si disponibles)
if len(incorrect_indices) > 0:
    for i, ax in enumerate(axes[1]):
        if i < len(incorrect_indices):
            idx = incorrect_indices[i]
            ax.imshow(X_test[idx], cmap='gray')
            confidence = np.max(y_pred_proba[idx])
            predicted_label = y_pred[idx]
            ax.set_title(f"Prédit: {predicted_label}\nVrai: {y_test[idx]}\nConf: {confidence:.2f}", 
                        fontsize=10, color='red')
        else:
            ax.axis('off')
        ax.axis('off')
    axes[1, 0].set_ylabel('Incorrects', fontsize=12, color='red')

plt.suptitle('Exemples de Prédictions - Modèle Keras', fontsize=14)
plt.tight_layout()
plt.savefig('prediction_examples.png', dpi=150)
plt.show()


## 💾 13. Sauvegarde et Export


In [ ]:
print("\n" + "="*70)
print("SAUVEGARDE DES RÉSULTATS")
print("="*70)

import json
import os

# Créer un dossier pour les résultats
os.makedirs('results', exist_ok=True)

# Sauvegarde du modèle
model.save('results/mnist_keras_model.h5')
print("✓ Modèle sauvegardé: results/mnist_keras_model.h5")

# Sauvegarde de l'historique
history_df = pd.DataFrame(history_dict)
history_df.to_csv('results/training_history.csv', index=False)
print("✓ Historique sauvegardé: results/training_history.csv")

# Sauvegarde des métriques
metrics = {
    'date': datetime.now().isoformat(),
    'hardware': {
        'cpu': platform.processor(),
        'cores': f"{psutil.cpu_count(logical=False)}P/{psutil.cpu_count(logical=True)}T",
        'ram_gb': ram.total // 1024**3,
        'python_version': platform.python_version(),
        'tensorflow_version': tf.__version__
    },
    'model': {
        'architecture': 'Flatten(784) → Dense(128) → Dense(64) → Dense(10)',
        'total_params': int(model.count_params()),
        'batch_size': batch_size,
        'epochs_trained': len(history_dict['accuracy'])
    },
    'performance': {
        'test_accuracy': float(test_acc),
        'test_loss': float(test_loss),
        'training_time': float(train_time),
        'best_val_accuracy': float(max(history_dict['val_accuracy'])) if 'val_accuracy' in history_dict else None,
        'epoch_to_97': int(epoch_to_97) if epoch_to_97 else None,
        'time_to_97': float(time_to_97) if epoch_to_97 else None,
        'inference_speed': float(inference_results['speed_img_per_sec'].iloc[-1])
    },
    'target_achievement': {
        'target': 0.97,
        'achieved': test_acc >= 0.97,
        'actual_percentage': float(test_acc * 100)
    }
}

with open('results/final_report.json', 'w', encoding='utf-8') as f:
    json.dump(metrics, f, indent=2, ensure_ascii=False)
print("✓ Rapport sauvegardé: results/final_report.json")

# Sauvegarde des images
print("✓ Images sauvegardées:")
print("  - training_curves.png")
print("  - confusion_matrix.png")
print("  - inference_performance.png")
print("  - model_comparison.png")
print("  - prediction_examples.png")


## 🏆 14. Conclusion et Recommandations


In [ ]:
print("\n" + "="*70)
print("CONCLUSION FINALE")
print("="*70)

print(f"\n🎯 OBJECTIF: Atteindre 97% d'accuracy sur MNIST rapidement")
print(f"📊 RÉSULTAT: {test_acc*100:.2f}% atteint en {train_time:.2f}s")

print("\n" + "="*70)
print("SYNTHÈSE DES RÉSULTATS")
print("="*70)

print(f"\n✅ SUCCÈS:")
print(f"1. Accuracy test: {test_acc*100:.2f}% (objectif: 97%)")
print(f"2. Temps entraînement: {train_time:.2f} secondes")
print(f"3. Vitesse inférence: {inference_results['speed_img_per_sec'].iloc[-1]:.0f} images/s")
print(f"4. Architecture: Simple mais efficace (2 couches cachées)")

print(f"\n📈 COMPARAISON AVEC AUTRES APPROCHES:")
print(f"• SGD Classifier: 91.26% en 15.70s (trop simple)")
print(f"• Random Forest: 96.81% en 37.88s (presque)")
print(f"• MLP Keras: {test_acc*100:.2f}% en {train_time:.2f}s (MEILLEUR)")

print(f"\n⚡ OPTIMISATIONS CLÉS:")
print("1. Batch Normalization → Convergence 2x plus rapide")
print("2. Dropout 25% → Réduction overfitting sans perte de capacité")
print("3. Adam optimizer → Meilleur que SGD pour MNIST")
print("4. Early Stopping → Évite le surapprentissage")

print(f"\n" + "="*70)
print("FIN DE L'ANALYSE")
print("="*70)

# Affichage final résumé
print(f"""
┌─────────────────────────────────────────────┐
│           RÉCAPITULATIF FINAL               │
├─────────────────────────────────────────────┤
│  Objectif:        97% accuracy              │
│  Résultat:        {test_acc*100:6.2f}%               │
│  Temps:           {train_time:6.2f}s               │
│  Modèle:          MLP Keras optimisé        │
│  Matériel:        i5-13420H (8 cores)       │
│  Status:          {'✅ SUCCÈS' if test_acc >= 0.97 else '⚠️ PRESQUE'}          │
└─────────────────────────────────────────────┘
""")
